In [4]:
import numpy as np
import quantecon as qe
def mc_sample_path(P, x0=None, sample_size=1000):

    # set up
    P = np.asarray(P)
    X = np.empty(sample_size, dtype=int)

    # Convert each row of P into a cdf
    n = len(P)
    P_dist = [np.cumsum(P[i, :]) for i in range(n)]

    # draw initial state, defaulting to 0
    if x0 is not None:
        X_0 = qe.random.draw(np.cumsum(x0))
    else:
        X_0 = 0

    # simulate
    X[0] = X_0
    for t in range(sample_size - 1):
        X[t+1] = qe.random.draw(P_dist[X[t]])

    return X

In [5]:
# time line
T_min = 15
T_max = 80

In [38]:
P = np.array([[0.4, 0.6],[0.2, 0.8]])
econState = mc_sample_path(P, sample_size=T_max-T_min+1)
econState

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [ ]:
# simulate an agent age 15 starting with wealth of 30
wealth = 30
Policy = Policy[::-1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def action(Policy, t, w, s):
    w_grid = np.linspace(w_lower, w_upper, num = w_grid_size)
    index = np.where(np.isclose(w_grid, w))
    if s == 0:
        p = Policy[t][:w_grid_size]
        return p[index]
    else
        p = Policy[t][w_grid_size:]
        return p[index]
    
# Define the transtiion of state 
def fixTransition(w,s,s_next, a):
    w, s = Z
    c, b, k = a
    # collect possible next state (w_next, s_next) with probability prob
    Z_next = []
    prob = []
    # depend on the current econ state s and future state s_next we have the following return on bond and stock 
    r_bond = r_f[int(s)]
    r_stock = r_m(s,s_next)
    w_next = b*(1+r_bond) + k*(1+r_stock)
    Z_next.append([w_next, s_next])
    prob.append(P[int(s), s_next])
    return w_next

In [ ]:
w = 30 
wealth = []
C = []
B = []
K = []
for t in range(len(econState)-1):
    wealth.append(w)
    s = econState[t]
    s_next = econState[t+1]
    a = policy(Policy, t, w, s)
    C.append(a[0])
    B.append(a[1])
    K.append(a[2])
    w = fixTransition(w,s,s_next, a)

In [6]:
%pylab inline
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from multiprocessing import Pool
from functools import partial 
import warnings
warnings.filterwarnings("ignore")
np.printoptions(precision=2)

# time line
T_min = 15
T_max = 80
T_y = 65

# States of the economy, GOOD or BAD, {1 : GOOD}, {0 : BAD}
S = [0,1]
# All the money amount are denoted in thousand dollars
earningShock = [0.9,1.1]
# Define transition matrix of economical states
# GOOD -> GOOD 0.8, BAD -> BAD 0.4 
P = np.array([[0.4, 0.6],[0.2, 0.8]])
# current risk free interest rate 
r_f = [0.01 ,0.03]
# stock return depends on current and future econ states
def r_m(current_state, next_state):
    if (current_state == 1 and next_state == 1):
        return 0.25
    elif (current_state == 1 and next_state == 0):
        return -0.05
    elif (current_state == 0 and next_state == 1):
        return 0.2
    else:
        return -0.1
    
#Define the utility function
def u(c):
    return np.log(c)
        

# Define deterministic function to captures the hump shape of life-cycle earnings.
ageTick = [15, 25, 35, 45, 55, 80]
earning = [35, 54, 67, 71, 60, 37]
cspline = interp1d(ageTick, earning, kind = "linear")
detEarning = np.array(cspline(np.linspace(15,80,66)))
def chi(age):
    return detEarning[int(age-T_min)]

def y(age, s):
    return chi(age) * earningShock[int(s)]

# Define the transtiion of state 
def transition(Z, a):
    '''
        Input: Z is a tuple containing states variables: (w, s) which is the current wealth and econ state
               a is the action taken with contains variables: (c, b, k) which is the combination of consumption, bond and stock
        Output: the next possible states with corresponding probabilities
    '''
    w, s = Z
    c, b, k = a
    # collect possible next state (w_next, s_next) with probability prob
    Z_next = []
    prob = []
    # depend on the current econ state s and future state s_next we have the following return on bond and stock 
    for s_next in S:
        r_bond = r_f[int(s)]
        r_stock = r_m(s,s_next)
        w_next = b*(1+r_bond) + k*(1+r_stock)
        Z_next.append([w_next, s_next])
        prob.append(P[s, s_next])
    return Z_next, prob

Populating the interactive namespace from numpy and matplotlib


In [7]:
# timeline is from age 15 - 80, use approximation for value function from age 15 - 79 by discretizing the state space
# since we know the terminal condition. Here XX is the grid point collection. Initiate the grid and corresponding value.

# This function is used to generate power grid 
def powspace(start, stop, power, num):
    start = np.power(start, 1/float(power))
    stop = np.power(stop, 1/float(power))
    return np.power(np.linspace(start, stop, num=num), power) 

w_grid_size = 1000
w_lower = 0
w_upper = 3000
T = T_max - T_min
# wgrid = powspace(w_lower, w_upper, 3, num = w_grid_size)
wgrid = np.linspace(w_lower, w_upper, num = w_grid_size)

print("Shape of grid w grid: ", wgrid.shape)
# V use to store the value 
V_t = np.zeros((w_grid_size,len(S),T))
C_t = np.zeros((w_grid_size,len(S),T))
B_t = np.zeros((w_grid_size,len(S),T))
K_t = np.zeros((w_grid_size,len(S),T))
print("Shape of Value matrix: ", V_t.shape)

# Calculate terminal value function 
V_t[:, 0, T-1] = u(y(T_max,0) + wgrid)
V_t[:, 1, T-1] = u(y(T_max,1) + wgrid)
C_t[:, 0, T-1] = y(T_max,0) + wgrid
C_t[:, 1, T-1] = y(T_max,1) + wgrid
B_t[:, 0, T-1] = 0
B_t[:, 1, T-1] = 0
K_t[:, 0, T-1] = 0
K_t[:, 1, T-1] = 0

Shape of grid w grid:  (1000,)
Shape of Value matrix:  (1000, 2, 65)


In [8]:
from scipy.optimize import minimize
# Definition of the value function, return of the funciton is the value of the function and the coresponding policy
def V(w, s, t, model):
    beta = 0.98
    def obj(bk):
        b, k = bk
        c = y(t,s) + w - b - k
        if c <= 0 or b <= 0 or k <= 0:
            return np.inf 
        Z_next, prob = transition([w,s], (c, b, k))
        V_next = np.array([model[z[1]](z[0]) for z in Z_next])
        return -(u(c) + beta * np.dot(V_next, prob))
    res = minimize(obj, [5,5],  method='SLSQP')
    return np.array([-res.fun, res.x])

In [31]:
def smooth(array):
    index = argwhere(np.isnan(array))
    n = len(array)
    for i in index:
        if i == 0:
            array[i] = array[i+1]
        elif i == len(array) - 1:
            array[i] = array[i-1]
        else:
            array[i] = (array[i-1] + array[i+1])/2
    return array

In [35]:
smooth(V_t[:,1,tt+1])

array([ 7.35831545,  7.43309502,  7.50636999,  7.58027676,  7.64606642,
        7.71752826,  7.78380687,  7.84601781,  7.90639383,  7.96514427,
        8.01818555,  8.07305749,  8.12495436,  8.17659631,  8.23348865,
        8.28474041,  8.33320788,  8.38063461,  8.4269938 ,  8.47259737,
        8.51153595,  8.55118412,  8.59499939,  8.63351985,  8.67425564,
        8.71393855,  8.75620473,  8.79298586,  8.83924222,  8.87579407,
        8.91191086,  8.94742665,  8.98231845,  9.01661435,  9.05031895,
        9.08347145,  9.11609933,  9.14836789,  9.18034076,  9.20305148,
        9.233464  ,  9.26051977,  9.29085646,  9.32175419,  9.34566203,
        9.37441607,  9.40257208,  9.4295965 ,  9.45662092,  9.45734466,
        9.48358968,  9.50949233,  9.53506036,  9.56030262,  9.58522704,
        9.60984153,  9.63415377,  9.65817117,  9.68190069,  9.70534932,
        9.72852326,  9.75142923,  9.77407318,  9.79646127,  9.81859877,
        9.84049168,  9.86214506,  9.88356427,  9.9047542 ,  9.92

In [9]:
# Set value to the grid by backward induction 
pool = Pool()

for t in range(T_max-2, T_min-1, -1):
    print(t)
    tt = t-T_min
    cs = [interp1d(wgrid, smooth(V_t[:,0,tt+1]), kind = "cubic", fill_value="extrapolate"),
          interp1d(wgrid, smooth(V_t[:,1,tt+1], kind = "cubic", fill_value="extrapolate")]    
    for s in S:                                               
        f = partial(V, s = s, t = t, model = cs)
        results = np.array(pool.map(f, wgrid))
        V_t[:,s,tt] = results[:,0]
        B_t[:,s,tt] = np.array([b[0] for b in results[:,1]])
        K_t[:,s,tt] = np.array([k[1] for k in results[:,1]])
        C_t[:,s,tt] = wgrid + y(t,s) - B_t[:,s,tt] - K_t[:,s,tt]
pool.close()

78
77


In [44]:
pool = Pool()

print(t)
tt = t-T_min
cs = [interp1d(wgrid, V_t[:,0,tt+1], kind = "cubic", fill_value="extrapolate"),
      interp1d(wgrid, V_t[:,1,tt+1], kind = "cubic", fill_value="extrapolate")]    
for s in S:                                               
    f = partial(V, s = s, t = t, model = cs)
    results = np.array(pool.map(f, wgrid))
    V_t[:,s,tt] = results[:,0]
    B_t[:,s,tt] = np.array([b[0] for b in results[:,1]])
    K_t[:,s,tt] = np.array([k[1] for k in results[:,1]])
    C_t[:,s,tt] = wgrid + y(t,s) - B_t[:,s,tt] - K_t[:,s,tt]
    
pool.close()

77


In [25]:
interp1d(wgrid, V_t[:,1,tt+1], kind = "cubic", fill_value="extrapolate")(wgrid)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na